In [ ]:
import datetime as dt
import pandas as pd

from fastcounting import helper, views, store, files, system, asum
%alias_magic t timeit

In [ ]:
import redis
r = redis.Redis(**helper.Helper().rediscred, decode_responses=True)

Check your local versions of pyredis and the redis-server you connected to:

In [ ]:
redis.__version__ # should be the latest from github repo in future it has to be equal or bigger to 3.4.2 

In [ ]:
r.info()['redis_version'] # redis server 6 beta is versioned as 5.9.xy , by the point of writing i am on 5.9.103

main pipeline can easily take 5 min for 200 000 transactions.

pd.read_excel takes about a third of that time.

In [ ]:
# read your cleaned journal data from file

In [ ]:
df, filename = files.main_etl('2017-13')

In [ ]:
# this takes around 5 mins

In [ ]:
store.main('2017-13')

In [ ]:
# diff

In [ ]:
asum.sum_account(0,9999999999999)

In [ ]:
asum.main_total_diff('2017-13')

In [ ]:
# datefilter

In [ ]:
start = dt.datetime(2017,2,2).timestamp()
end = dt.datetime(2017,8,3).timestamp()

views.main_datefilter(start, end)

In [ ]:
# push your account system to redis database

In [ ]:
system.accountsystem_toredis()

In [ ]:
# example for date and accountfilter which returns a set which is the intersections of those two

In [ ]:
def filter(start_date, end_date, start_account, end_account):
    date_filter = r.zrangebyscore('atomic:date', start_date, end_date)
    account_filter = r.zrangebyscore('account:atomic', start_account, end_account)
    atomics = set.intersection(set(date_filter), set(account_filter))
    return atomics

In [ ]:
start = dt.datetime(2017,2,2).timestamp()
end = dt.datetime(2017,4,3).timestamp()

atomics = filter(start, end, 0, 99999)